# LCA with uncertainty

This notebook demonstrates how to perform uncertainty analysis using this library. It covers two sections, with each section showing how to add uncertainty in different ways: itemwise, columnwise, or uniformly, and different matrices technosphere and biosphere:  
- Section 1: Uncertainty analysis with only background database  
- Section 2: Uncertainty analysis with both foreground and background database  

Uncertainty strategy:
1) `uniformly`: This strategy add the same type/value of the uncertainty to a matrix, but you can add uncertainty for all matricies or only one/two of the matrices.
2) `columnwise`: This strategy add the same type/value of the uncertainty to the same column of a matrix, different column can have different type/value of uncertainty. The uncertainty first save into metadata and then add to the uncertainty array accordingly. To use this stragety, your uncertainty input should be defined in the file or save it directly to metadata of this library.
3) `itemwise`: This strategy add different type/value of the uncertainty different element in the matrix. To use this stragety, your uncertainty input should be defined in the file or save it directly to metadata of this library.

Note: for strategy 2) and 3), only technosphere and biosphere matrices are supported.


## Preparation

1. Import all needed libraries

In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))
from bw_bamboo.background_importer import *
from bw_bamboo.foreground_importer import *
from bw_bamboo.datapackage_builder import *
from bw_bamboo.uncertainty_handler import *
from bw_bamboo.lca_wrapper import *

2. Define some constants for this notebook

In [2]:
GHG = ["CO2 - combustion - air",
        "CO2 - non combustion - Cement production - air",
        "CO2 - non combustion - Lime production - air",
        "CO2 - waste - biogenic - air", 
        "CO2 - waste - fossil - air",
        "CO2 - agriculture - peat decay - air", 
        "CH4 - agriculture - air",
        "CH4 - waste - air",
        "CH4 - combustion - air",
        "CH4 - non combustion - Extraction/production of (natural) gas - air",
        "CH4 - non combustion - Extraction/production of crude oil - air",
        "CH4 - non combustion - Mining of antracite - air",
        "CH4 - non combustion - Mining of bituminous coal - air",
        "CH4 - non combustion - Mining of coking coal - air",
        "CH4 - non combustion - Mining of lignite (brown coal) - air",
        "CH4 - non combustion - Mining of sub-bituminous coal - air",
        "CH4 - non combustion - Oil refinery - air",
        "N2O - combustion - air",
        "N2O - agriculture - air",
        "SF6 - air"]

METHOD = ('IPCC 2013', 'climate change', 'global warming potential (GWP100)')
FG_MAPPING_FILE = os.path.join(os.getcwd(), "data", "EXIOBASE-ecoinvent-bio-bw-GHG.csv")
EXIOBASE_AGGREGATED_A_FILE = os.path.join(os.getcwd(), "data/A.txt")
EXIOBASE_AGGREGATED_S_FILE = os.path.join(os.getcwd(), "data/S.txt")
EMISSION_CODE_FILE = os.path.join(os.getcwd(), "data/EXIOBASE-ecoinvent-bio-bw-GHG.csv")
FROEGROUND_1 = os.path.join(os.getcwd(), "data/20250110_foreground_system_small.csv")
# FROEGROUND_1 = os.path.join(os.getcwd(), "data/fg_exiobase_aggregated_1col.csv")
FROEGROUND_2 = os.path.join(os.getcwd(), "data/fg_exiobase_aggregated_2col.csv")
mysector = 68  # TODO: why the first one is minus, 68 looks normal.

## Section 1: Uncertainty analysis with only background database

So, before everything start, here is a note: 
1. In the uncertainty analysis, we need get the matricies data in datapackage needed format first. 
2. Then we are able to get the uncertainty array in datapackage needed format. 
3. And finally, we call function `prepare_datapackage`, it will return the corresponding datapackage.

1. Get technosphere matrix

In [3]:
bg_importer = BackgroundImporter()

tech_df = pd.read_table(EXIOBASE_AGGREGATED_A_FILE, sep='\t', header=None, low_memory=False)
raw_tech = tech_df.iloc[3:, 2:].astype('float').to_numpy()
bg_tech_matrix = bg_importer.form_tech_matrix(raw_tech)

2. Get biosphere matrix

In [4]:
bio_df = pd.read_csv(EXIOBASE_AGGREGATED_S_FILE, header=[0,1], index_col=[0], sep='\t', low_memory=False)
bg_bio_matrix = bg_importer.form_bio_matrix(bio_df, GHG)

3. Get characterization factor matrix

In [5]:
# cf_matrix = bg_importer.form_cf_matrix("ecoinvent-3.11-biosphere", METHOD, FG_MAPPING_FILE, GHG)
# cf_matrix.diagonal()  # print the diagnal for checking

# since 2 of the emissions we chose has no cf value in IPCC2013 method, we assign the values so that we can use.
cf_matrix = [1., 1., 1., 1., 1., 1., 27.0, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 273., 273., 25200.]
bg_cf_matrix = np.diagflat(cf_matrix)

### 1) Add uncertainty uniformly.

1. prepare datapackage data, including matrix data, datapackage indices and datapackage flip.

In [6]:
dp_builder = DatapackageBuilder()
(bg_tech_data, bg_tech_indices, bg_tech_flip), (bg_bio_data, bg_bio_indices), (bg_cf_data, bg_cf_indices) = dp_builder.prepare_dp_matrix(bg_tech_matrix, bg_bio_matrix, bg_cf_matrix)

2. Add uncertainty uniformly

In [7]:
uncertainty_handler = UncertaintyHandler()

uncertainty_type = 2 # attention: variable cannot be called "type", it conflict with the Python function type()
bg_tech_uncertainty_array = uncertainty_handler.add_uniform_uncertainty(uncertainty_type, 1.1, bg_tech_data, bg_tech_flip)
bg_bio_uncertainty_array = uncertainty_handler.add_uniform_uncertainty(uncertainty_type, 1.1, bg_bio_data, None)
bg_cf_uncertainty_array = uncertainty_handler.add_uniform_uncertainty(uncertainty_type, 1.1, bg_cf_data, None)

# Here, we check the shape of the uncertainty array. As long as the data shape is identical to the uncertainty shape, it is correct.
# Note: the shape is not identical to the matrix shape (76 * 76), (20 * 76) or (20), because a sparse matrix does not store zero data.
print(bg_tech_data.shape, bg_bio_data.shape, bg_cf_data.shape)
print(bg_tech_uncertainty_array.shape, bg_bio_uncertainty_array.shape, bg_cf_uncertainty_array.shape)
print(bg_tech_uncertainty_array)  # check the uncertainties are correct.

(5506,) (1405,) (20,)
(5506,) (1405,) (20,)
[(0,  0.7363383,        nan, nan, nan, nan, False)
 (2, -6.6262465, 0.09531018, nan, nan, nan, False)
 (2, -5.9899883, 0.09531018, nan, nan, nan, False) ...
 (2, -5.0765004, 0.09531018, nan, nan, nan, False)
 (2, -6.968573 , 0.09531018, nan, nan, nan, False)
 (0,  0.9951783,        nan, nan, nan, nan, False)]


3. Create datapackage

In [8]:
bg_dp = dp_builder.prepare_datapackage([(bg_tech_data, bg_tech_indices, bg_tech_flip), (bg_bio_data, bg_bio_indices), (bg_cf_data, bg_cf_indices)], [bg_tech_uncertainty_array, bg_bio_uncertainty_array, bg_cf_uncertainty_array])

4. Run simulation

In [9]:
bg_activities = get_bg_activities(EXIOBASE_AGGREGATED_A_FILE, "\t")

lca = bc.LCA(
            demand={mysector : 1},
            data_objs=[bg_dp],
            use_distributions=True,
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, bg_activities[mysector]}")

Brightway calculated lca score: (301874.9383870075, 'RoW-Services')


### 2) Add uncertainty columnwise.

When we add uncertainty columnwise, we use metadata to save all the uncertainty values and relavant information.

1. Define the columnwise uncertainty information

In [10]:
# Create a random uncertainty list as the uncertainty list for this exaample.
uncertainty_type = 2
uncertainty_list = [ (i + 1) * 1.01 for i in range(len(bg_tech_matrix))]  # the uncertainties for technosphere and biosphere matrices

2. Save all activities to metadata

In [11]:
bg_activities

for i in range(len(bg_activities)):
    uncertainty_handler.metadata[i] = {"act": bg_activities[i]}

uncertainty_handler.metadata # now we have all activity names and indices saved in the metadata.

{0: {'act': 'EU28-Agriculture-Forestry-Fishing'},
 1: {'act': 'EU28-Energy'},
 2: {'act': 'EU28-Natural gas and services related to natural gas extraction, excluding surveying'},
 3: {'act': 'EU28-Industry'},
 4: {'act': 'EU28-Motor Gasoline'},
 5: {'act': 'EU28-Gas/Diesel Oil'},
 6: {'act': 'EU28-Refinery Feedstocks'},
 7: {'act': 'EU28-Lubricants'},
 8: {'act': 'EU28-P- and other fertiliser'},
 9: {'act': 'EU28-Chemicals nec'},
 10: {'act': 'EU28-Biogasoline'},
 11: {'act': 'EU28-Biodiesels'},
 12: {'act': 'EU28-Cement, lime and plaster'},
 13: {'act': 'EU28-Basic iron and steel and of ferro-alloys and first products thereof'},
 14: {'act': 'EU28-Aluminium and aluminium products'},
 15: {'act': 'EU28-Copper products'},
 16: {'act': 'EU28-Electricity by coal'},
 17: {'act': 'EU28-Electricity by gas'},
 18: {'act': 'EU28-Electricity by nuclear'},
 19: {'act': 'EU28-Electricity by hydro'},
 20: {'act': 'EU28-Electricity by wind'},
 21: {'act': 'EU28-Electricity by petroleum and other oi

3. Save columnwise uncertainty values and uncertainty types (in this example, we only define 1 type in total)

In [12]:
for i, j in zip(range(len(bg_activities)), uncertainty_list):
    uncertainty_handler.metadata[i]["gsd"] = j
    uncertainty_handler.metadata[i]["type"] = uncertainty_type
uncertainty_handler.metadata # now we save one uncertainty for one column in the metadata.

{0: {'act': 'EU28-Agriculture-Forestry-Fishing', 'gsd': 1.01, 'type': 2},
 1: {'act': 'EU28-Energy', 'gsd': 2.02, 'type': 2},
 2: {'act': 'EU28-Natural gas and services related to natural gas extraction, excluding surveying',
  'gsd': 3.0300000000000002,
  'type': 2},
 3: {'act': 'EU28-Industry', 'gsd': 4.04, 'type': 2},
 4: {'act': 'EU28-Motor Gasoline', 'gsd': 5.05, 'type': 2},
 5: {'act': 'EU28-Gas/Diesel Oil', 'gsd': 6.0600000000000005, 'type': 2},
 6: {'act': 'EU28-Refinery Feedstocks', 'gsd': 7.07, 'type': 2},
 7: {'act': 'EU28-Lubricants', 'gsd': 8.08, 'type': 2},
 8: {'act': 'EU28-P- and other fertiliser', 'gsd': 9.09, 'type': 2},
 9: {'act': 'EU28-Chemicals nec', 'gsd': 10.1, 'type': 2},
 10: {'act': 'EU28-Biogasoline', 'gsd': 11.11, 'type': 2},
 11: {'act': 'EU28-Biodiesels', 'gsd': 12.120000000000001, 'type': 2},
 12: {'act': 'EU28-Cement, lime and plaster', 'gsd': 13.13, 'type': 2},
 13: {'act': 'EU28-Basic iron and steel and of ferro-alloys and first products thereof',
  '

4. Get matrix data in datapackage needed format

In [13]:
(bg_tech_data, bg_tech_indices, bg_tech_flip), (bg_bio_data, bg_bio_indices), (bg_cf_data, bg_cf_indices) = dp_builder.prepare_dp_matrix(bg_tech_matrix, bg_bio_matrix, bg_cf_matrix)

5. Form uncertainties into datapackage needed format.

In [14]:
bg_tech_uncertainty_array = uncertainty_handler.add_ununiform_uncertainty(bg_tech_data, bg_tech_indices, bg_tech_flip, "columnwise")
bg_bio_uncertainty_array = uncertainty_handler.add_ununiform_uncertainty(bg_bio_data, bg_bio_indices, None, "columnwise")

# Here, we check the shape of the uncertainty array. As long as the data shape is identical to the uncertainty shape, it is correct.
# Note: the shape is not identical to the matrix shape (76 * 76), (20 * 76) or (20), because a sparse matrix does not store zero data.
print(bg_tech_data.shape, bg_bio_data.shape)
print(bg_tech_uncertainty_array.shape, bg_bio_uncertainty_array.shape)
print(bg_tech_uncertainty_array)  # check the uncertainties are correct.

(5506,) (1405,)
(5506,) (1405,)
[(2, -3.0606556e-01, 0.00995033, nan, nan, nan, False)
 (2, -6.6262465e+00, 0.7030975 , nan, nan, nan, False)
 (2, -5.9899883e+00, 1.1085626 , nan, nan, nan, False) ...
 (2, -5.0765004e+00, 4.3140154 , nan, nan, nan, False)
 (2, -6.9685731e+00, 4.3274384 , nan, nan, nan, False)
 (2, -4.8333639e-03, 4.3406835 , nan, nan, nan, False)]


6. Create datapackage

In [15]:

dp_2 = dp_builder.prepare_datapackage([(bg_tech_data, bg_tech_indices, bg_tech_flip), (bg_bio_data, bg_bio_indices), (bg_cf_data, bg_cf_indices)], [bg_tech_uncertainty_array, bg_bio_uncertainty_array, None])
dp_2.data

[array([( 0,  0), ( 0,  1), ( 0,  2), ..., (75, 73), (75, 74), (75, 75)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([7.36338343e-01, 1.32512793e-03, 2.50369329e-03, ...,
        6.24171469e-03, 9.40994608e-04, 9.95178298e-01]),
 array([(2, -3.0606556e-01, 0.00995033, nan, nan, nan, False),
        (2, -6.6262465e+00, 0.7030975 , nan, nan, nan, False),
        (2, -5.9899883e+00, 1.1085626 , nan, nan, nan, False), ...,
        (2, -5.0765004e+00, 4.3140154 , nan, nan, nan, False),
        (2, -6.9685731e+00, 4.3274384 , nan, nan, nan, False),
        (2, -4.8333639e-03, 4.3406835 , nan, nan, nan, False)],
       dtype=[('uncertainty_type', 'u1'), ('loc', '<f4'), ('scale', '<f4'), ('shape', '<f4'), ('minimum', '<f4'), ('maximum', '<f4'), ('negative', '?')]),
 array([False,  True,  True, ...,  True,  True, False]),
 array([(76,  0), (76,  1), (76,  2), ..., (95, 73), (95, 74), (95, 75)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([6.92719535e+04, 2.24902521e+05, 1

5. Run simulation

In [16]:
lca = bc.LCA(
            demand={mysector : 1},
            data_objs=[dp_2],
            use_distributions=True,
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, bg_activities[mysector]}")

Brightway calculated lca score: (35407858.33583058, 'RoW-Services')


### 3) Add uncertainty itemwise.

When we add uncertainty columnwise, we use metadata to save all the uncertainty values and relavant information.

1. Define the columnwise uncertainty information

In [17]:
# Create a random uncertainty list as the uncertainty list for this exaample.
uncertainty_type = 2
x = len(bg_tech_matrix)
uncertainty_matrix = [[(i + 1) * 1.01 for i in range(x)] for _ in range(x)]

2. Save all activities to metadata

In [18]:
for i in range(len(bg_activities)):
    uncertainty_handler.metadata[i] = {"act": bg_activities[i]}

uncertainty_handler.metadata # now we have all activity names and indices saved in the metadata.

{0: {'act': 'EU28-Agriculture-Forestry-Fishing'},
 1: {'act': 'EU28-Energy'},
 2: {'act': 'EU28-Natural gas and services related to natural gas extraction, excluding surveying'},
 3: {'act': 'EU28-Industry'},
 4: {'act': 'EU28-Motor Gasoline'},
 5: {'act': 'EU28-Gas/Diesel Oil'},
 6: {'act': 'EU28-Refinery Feedstocks'},
 7: {'act': 'EU28-Lubricants'},
 8: {'act': 'EU28-P- and other fertiliser'},
 9: {'act': 'EU28-Chemicals nec'},
 10: {'act': 'EU28-Biogasoline'},
 11: {'act': 'EU28-Biodiesels'},
 12: {'act': 'EU28-Cement, lime and plaster'},
 13: {'act': 'EU28-Basic iron and steel and of ferro-alloys and first products thereof'},
 14: {'act': 'EU28-Aluminium and aluminium products'},
 15: {'act': 'EU28-Copper products'},
 16: {'act': 'EU28-Electricity by coal'},
 17: {'act': 'EU28-Electricity by gas'},
 18: {'act': 'EU28-Electricity by nuclear'},
 19: {'act': 'EU28-Electricity by hydro'},
 20: {'act': 'EU28-Electricity by wind'},
 21: {'act': 'EU28-Electricity by petroleum and other oi

3. Save itemwise uncertainty values and uncertainty types (in this example, we only define 1 type in total)

In [46]:
for i, j in zip(range(len(bg_activities)), uncertainty_matrix):
    uncertainty_handler.metadata[i]["specific"] = j  # here "j" is a list
    uncertainty_handler.metadata[i]["type"] = uncertainty_type
uncertainty_handler.metadata[0] # now we save one uncertainty for one activity in the metadata.

{'act': 'column_1',
 'specific': [1.01,
  2.02,
  3.0300000000000002,
  4.04,
  5.05,
  6.0600000000000005,
  7.07,
  8.08,
  9.09,
  10.1,
  11.11,
  12.120000000000001,
  13.13,
  14.14,
  15.15,
  16.16,
  17.17,
  18.18,
  19.19,
  20.2,
  21.21,
  22.22,
  23.23,
  24.240000000000002,
  25.25,
  26.26,
  27.27,
  28.28,
  29.29,
  30.3,
  31.31,
  32.32,
  33.33,
  34.34,
  35.35,
  36.36,
  37.37,
  38.38,
  39.39,
  40.4,
  41.410000000000004,
  42.42,
  43.43,
  44.44,
  45.45,
  46.46,
  47.47,
  48.480000000000004,
  49.49,
  50.5,
  51.51,
  52.52,
  53.53,
  54.54,
  55.55,
  56.56,
  57.57,
  58.58,
  59.59,
  60.6,
  61.61,
  62.62,
  63.63,
  64.64,
  65.65,
  66.66,
  67.67,
  68.68,
  69.69,
  70.7,
  71.71,
  72.72,
  73.73,
  74.74,
  75.75,
  76.76,
  77.77,
  78.78],
 'type': 2}

4. Get matrix data in datapackage needed format

In [20]:
(bg_tech_data, bg_tech_indices, bg_tech_flip), (bg_bio_data, bg_bio_indices), (bg_cf_data, bg_cf_indices) = dp_builder.prepare_dp_matrix(bg_tech_matrix, bg_bio_matrix, bg_cf_matrix)

5. Form uncertainties into datapackage needed format.

Here we only add uncertainty to technosphere matrix, if you want to add also to biosphere, you can save the uncertainty data to metadata first, and use the same way to add uncertainty for biosphere.

In [21]:
bg_tech_uncertainty_array = uncertainty_handler.add_ununiform_uncertainty(bg_tech_data, bg_tech_indices, bg_tech_flip, "itemwise")

6. Create datapackage

In [22]:
bg_dp_2 = dp_builder.prepare_datapackage([(bg_tech_data, bg_tech_indices, bg_tech_flip), (bg_bio_data, bg_bio_indices), (bg_cf_data, bg_cf_indices)], [bg_tech_uncertainty_array, None, None])
bg_dp_2.data

[array([( 0,  0), ( 0,  1), ( 0,  2), ..., (75, 73), (75, 74), (75, 75)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([7.36338343e-01, 1.32512793e-03, 2.50369329e-03, ...,
        6.24171469e-03, 9.40994608e-04, 9.95178298e-01]),
 array([(2, -3.0606556e-01, 0.00995033, nan, nan, nan, False),
        (2, -6.6262465e+00, 0.00995033, nan, nan, nan, False),
        (2, -5.9899883e+00, 0.00995033, nan, nan, nan, False), ...,
        (2, -5.0765004e+00, 4.3406835 , nan, nan, nan, False),
        (2, -6.9685731e+00, 4.3406835 , nan, nan, nan, False),
        (2, -4.8333639e-03, 4.3406835 , nan, nan, nan, False)],
       dtype=[('uncertainty_type', 'u1'), ('loc', '<f4'), ('scale', '<f4'), ('shape', '<f4'), ('minimum', '<f4'), ('maximum', '<f4'), ('negative', '?')]),
 array([False,  True,  True, ...,  True,  True, False]),
 array([(76,  0), (76,  1), (76,  2), ..., (95, 73), (95, 74), (95, 75)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([6.92719535e+04, 2.24902521e+05, 1

5. Run simulation

In [23]:
lca = bc.LCA(
            demand={mysector : 1},
            data_objs=[bg_dp_2],
            use_distributions=True,
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, bg_activities[mysector]}")

Brightway calculated lca score: (688394.0397545991, 'RoW-Services')


## Section 2: Uncertainty analysis with both foreground and background database

1. Read the foreground input file (we use the file with 2 foreground activities)

In [24]:
fg_df = pd.read_csv(FROEGROUND_2, decimal=",")
fg_df.head()

,Activity name,Exchange name,Exchange type,Exchange amount,Exchange uncertainty type,GSD,Exchange negative
0,column_1,EU28-Electricity by biomass and waste,technosphere,-2.34E-07,2,2.674794737,True
1,column_1,EU28-Electricity by solar photovoltaic,technosphere,-2.01E-07,2,2.674794737,True
2,column_1,EU28-Electricity by solar thermal,technosphere,0,1,0,False
3,column_1,"EU28-Electricity by tide, wave, ocean",technosphere,-6.78E-09,2,2.674794737,True
4,column_1,EU28-Electricity by Geothermal,technosphere,-9.35E-10,2,2.674794737,True


2. Get foreground matrices

In [25]:
fg_importer = ForegroundImporter()
fg_activities = get_fg_activities(FROEGROUND_2, ",")
fgbg, fgfg, bgfg, bifg = fg_importer.extend_matrix(fg_df, GHG, fg_activities, bg_activities)

3. Concatenate foreground with background

In [26]:
fg_bg_tech_matrix, fg_bg_bio_matrix = fg_importer.concatenate_matrix(bg_tech_matrix, bg_bio_matrix, fgbg, fgfg, bgfg, bifg)

# to compare the matrix before and after concatenate with foreground system
print(f"Technosphere shape change to {fg_bg_tech_matrix.shape} from {bg_tech_matrix.shape}")
print(f"Biosphere shape change to {fg_bg_bio_matrix.shape} from {bg_bio_matrix.shape}")

Technosphere shape change to (78, 78) from (76, 76)
Biosphere shape change to (20, 78) from (20, 76)


### 1) Add uncertainty uniformly (both foreground and background)

1. prepare datapackage data, including matrix data, datapackage indices and datapackage flip.

In [27]:
dp_builder = DatapackageBuilder()
(fg_bg_tech_data, fg_bg_tech_indices, fg_bg_tech_flip), (fg_bg_bio_data, fg_bg_bio_indices), (fg_bg_cf_data, fg_bg_cf_indices) = dp_builder.prepare_dp_matrix(fg_bg_tech_matrix, fg_bg_bio_matrix, bg_cf_matrix)

2. Add uncertainty uniformly

In [28]:
uncertainty_handler = UncertaintyHandler()

uncertainty_type = 2 # attention: variable cannot be called "type", it conflict with the Python function type()
fg_bg_tech_uncertainty_array = uncertainty_handler.add_uniform_uncertainty(uncertainty_type, 1.01, fg_bg_tech_data, fg_bg_tech_flip)
fg_bg_bio_uncertainty_array = uncertainty_handler.add_uniform_uncertainty(uncertainty_type, 1.01, fg_bg_bio_data, None)
fg_bg_cf_uncertainty_array = uncertainty_handler.add_uniform_uncertainty(uncertainty_type, 1.01, fg_bg_cf_data, None)

# Here, we check the shape of the uncertainty array. As long as the data shape is identical to the uncertainty shape, it is correct.
# Note: the shape is not identical to the matrix shape (76 * 76), (20 * 76) or (20), because a sparse matrix does not store zero data.
print(fg_bg_tech_data.shape, fg_bg_bio_data.shape, fg_bg_cf_data.shape)
print(fg_bg_tech_uncertainty_array.shape, fg_bg_bio_uncertainty_array.shape, fg_bg_cf_uncertainty_array.shape)
print(fg_bg_tech_uncertainty_array.shape)  # check the uncertainties are correct.

(5524,) (1407,) (20,)
(5524,) (1407,) (20,)
(5524,)


3. Create datapackage

In [29]:
bg_dp = dp_builder.prepare_datapackage([(fg_bg_tech_data, fg_bg_tech_indices, fg_bg_tech_flip), (fg_bg_bio_data, fg_bg_bio_indices), (fg_bg_cf_data, fg_bg_cf_indices)], [fg_bg_tech_uncertainty_array, fg_bg_bio_uncertainty_array, fg_bg_cf_uncertainty_array])

4. Run simulation

In [30]:
bg_activities = get_bg_activities(EXIOBASE_AGGREGATED_A_FILE, "\t")
fg_activities = get_fg_activities(FROEGROUND_2, "\t")

activities = fg_activities + bg_activities

lca = bc.LCA(
            demand={mysector : 1},
            data_objs=[bg_dp],
            use_distributions=True,
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, activities[mysector]}")

Brightway calculated lca score: (51530509.8442802, 'RoW-Steam and hot water supply services')


### 2) Add uncertainty columnwise.

When we add uncertainty columnwise, we need to use metadata to save all the uncertainty values and relavant information.

1. Define the columnwise uncertainty information

In [31]:
# Create a random uncertainty list as the uncertainty list for this exaample.
uncertainty_type = 2
uncertainty_list = [ (i + 1) * 1.01 for i in range(len(fg_bg_tech_matrix))]  # the uncertainties for technosphere and biosphere matrices

2. Save all activities to metadata

In [32]:
activities = fg_activities + bg_activities

for i in range(len(activities)):
    uncertainty_handler.metadata[i] = {"act": activities[i]}

uncertainty_handler.metadata # now we have all activity names and indices saved in the metadata.

{0: {'act': 'column_1'},
 1: {'act': 'column_2'},
 2: {'act': 'EU28-Agriculture-Forestry-Fishing'},
 3: {'act': 'EU28-Energy'},
 4: {'act': 'EU28-Natural gas and services related to natural gas extraction, excluding surveying'},
 5: {'act': 'EU28-Industry'},
 6: {'act': 'EU28-Motor Gasoline'},
 7: {'act': 'EU28-Gas/Diesel Oil'},
 8: {'act': 'EU28-Refinery Feedstocks'},
 9: {'act': 'EU28-Lubricants'},
 10: {'act': 'EU28-P- and other fertiliser'},
 11: {'act': 'EU28-Chemicals nec'},
 12: {'act': 'EU28-Biogasoline'},
 13: {'act': 'EU28-Biodiesels'},
 14: {'act': 'EU28-Cement, lime and plaster'},
 15: {'act': 'EU28-Basic iron and steel and of ferro-alloys and first products thereof'},
 16: {'act': 'EU28-Aluminium and aluminium products'},
 17: {'act': 'EU28-Copper products'},
 18: {'act': 'EU28-Electricity by coal'},
 19: {'act': 'EU28-Electricity by gas'},
 20: {'act': 'EU28-Electricity by nuclear'},
 21: {'act': 'EU28-Electricity by hydro'},
 22: {'act': 'EU28-Electricity by wind'},
 23:

3. Save columnwise uncertainty values and uncertainty types (in this example, we only define 1 type in total)

In [33]:
for i, j in zip(range(len(activities)), uncertainty_list):
    uncertainty_handler.metadata[i]["gsd"] = j
    uncertainty_handler.metadata[i]["type"] = uncertainty_type
uncertainty_handler.metadata # now we save one uncertainty for one column in the metadata.

{0: {'act': 'column_1', 'gsd': 1.01, 'type': 2},
 1: {'act': 'column_2', 'gsd': 2.02, 'type': 2},
 2: {'act': 'EU28-Agriculture-Forestry-Fishing',
  'gsd': 3.0300000000000002,
  'type': 2},
 3: {'act': 'EU28-Energy', 'gsd': 4.04, 'type': 2},
 4: {'act': 'EU28-Natural gas and services related to natural gas extraction, excluding surveying',
  'gsd': 5.05,
  'type': 2},
 5: {'act': 'EU28-Industry', 'gsd': 6.0600000000000005, 'type': 2},
 6: {'act': 'EU28-Motor Gasoline', 'gsd': 7.07, 'type': 2},
 7: {'act': 'EU28-Gas/Diesel Oil', 'gsd': 8.08, 'type': 2},
 8: {'act': 'EU28-Refinery Feedstocks', 'gsd': 9.09, 'type': 2},
 9: {'act': 'EU28-Lubricants', 'gsd': 10.1, 'type': 2},
 10: {'act': 'EU28-P- and other fertiliser', 'gsd': 11.11, 'type': 2},
 11: {'act': 'EU28-Chemicals nec', 'gsd': 12.120000000000001, 'type': 2},
 12: {'act': 'EU28-Biogasoline', 'gsd': 13.13, 'type': 2},
 13: {'act': 'EU28-Biodiesels', 'gsd': 14.14, 'type': 2},
 14: {'act': 'EU28-Cement, lime and plaster', 'gsd': 15.15

4. Get matrix data in datapackage needed format

In [34]:
(fg_bg_tech_data, fg_bg_tech_indices, fg_bg_tech_flip), (fg_bg_bio_data, fg_bg_bio_indices), (bg_cf_data, bg_cf_indices) = dp_builder.prepare_dp_matrix(fg_bg_tech_matrix, fg_bg_bio_matrix, bg_cf_matrix)

5. Form uncertainties into datapackage needed format.

In [35]:
fg_bg_tech_uncertainty_array = uncertainty_handler.add_ununiform_uncertainty(fg_bg_tech_data, fg_bg_tech_indices, fg_bg_tech_flip, "columnwise")
fg_bg_bio_uncertainty_array = uncertainty_handler.add_ununiform_uncertainty(fg_bg_bio_data, fg_bg_bio_indices, None, "columnwise")

# Here, we check the shape of the uncertainty array. As long as the data shape is identical to the uncertainty shape, it is correct.
# Note: the shape is not identical to the matrix shape (76 * 76), (20 * 76) or (20), because a sparse matrix does not store zero data.
print(fg_bg_tech_data.shape, fg_bg_bio_data.shape)
print(fg_bg_tech_uncertainty_array.shape, fg_bg_bio_uncertainty_array.shape)
print(fg_bg_tech_uncertainty_array)  # check the uncertainties are correct.

(5524,) (1407,)
(5524,) (1407,)
[(2,  0.0000000e+00, 0.00995033, nan, nan, nan, False)
 (2,  0.0000000e+00, 0.7030975 , nan, nan, nan, False)
 (2, -3.0606556e-01, 1.1085626 , nan, nan, nan, False) ...
 (2, -5.0765004e+00, 4.3406835 , nan, nan, nan, False)
 (2, -6.9685731e+00, 4.353756  , nan, nan, nan, False)
 (2, -4.8333639e-03, 4.366659  , nan, nan, nan, False)]


6. Create datapackage

In [36]:

dp_2 = dp_builder.prepare_datapackage([(fg_bg_tech_data, fg_bg_tech_indices, fg_bg_tech_flip), (fg_bg_bio_data, fg_bg_bio_indices), (bg_cf_data, bg_cf_indices)], [fg_bg_tech_uncertainty_array, fg_bg_bio_uncertainty_array, None])
dp_2.data

[array([( 0,  0), ( 1,  1), ( 2,  2), ..., (77, 75), (77, 76), (77, 77)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([1.00000000e+00, 1.00000000e+00, 7.36338343e-01, ...,
        6.24171469e-03, 9.40994608e-04, 9.95178298e-01]),
 array([(2,  0.0000000e+00, 0.00995033, nan, nan, nan, False),
        (2,  0.0000000e+00, 0.7030975 , nan, nan, nan, False),
        (2, -3.0606556e-01, 1.1085626 , nan, nan, nan, False), ...,
        (2, -5.0765004e+00, 4.3406835 , nan, nan, nan, False),
        (2, -6.9685731e+00, 4.353756  , nan, nan, nan, False),
        (2, -4.8333639e-03, 4.366659  , nan, nan, nan, False)],
       dtype=[('uncertainty_type', 'u1'), ('loc', '<f4'), ('scale', '<f4'), ('shape', '<f4'), ('minimum', '<f4'), ('maximum', '<f4'), ('negative', '?')]),
 array([False, False, False, ...,  True,  True, False]),
 array([(78,  2), (78,  3), (78,  4), ..., (97, 75), (97, 76), (97, 77)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([6.92719535e+04, 2.24902521e+05, 1

5. Run simulation

In [37]:
lca = bc.LCA(
            demand={mysector : 1},
            data_objs=[dp_2],
            use_distributions=True,
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, bg_activities[mysector]}")

Brightway calculated lca score: (7547047721.753698, 'RoW-Services')


### 3) Add uncertainty itemwise.

When we add uncertainty itemwise, we need to use metadata to save all the uncertainty values and relavant information.

1. Define the columnwise uncertainty information

In [38]:
# Create a random uncertainty list as the uncertainty list for this exaample.
uncertainty_type = 2
x = len(fg_bg_tech_matrix)
uncertainty_matrix = [[(i + 1) * 1.01 for i in range(x)] for _ in range(x)]

2. Save all activities to metadata

In [39]:
for i in range(len(activities)):
    uncertainty_handler.metadata[i] = {"act": activities[i]}

uncertainty_handler.metadata # now we have all activity names and indices saved in the metadata.

{0: {'act': 'column_1'},
 1: {'act': 'column_2'},
 2: {'act': 'EU28-Agriculture-Forestry-Fishing'},
 3: {'act': 'EU28-Energy'},
 4: {'act': 'EU28-Natural gas and services related to natural gas extraction, excluding surveying'},
 5: {'act': 'EU28-Industry'},
 6: {'act': 'EU28-Motor Gasoline'},
 7: {'act': 'EU28-Gas/Diesel Oil'},
 8: {'act': 'EU28-Refinery Feedstocks'},
 9: {'act': 'EU28-Lubricants'},
 10: {'act': 'EU28-P- and other fertiliser'},
 11: {'act': 'EU28-Chemicals nec'},
 12: {'act': 'EU28-Biogasoline'},
 13: {'act': 'EU28-Biodiesels'},
 14: {'act': 'EU28-Cement, lime and plaster'},
 15: {'act': 'EU28-Basic iron and steel and of ferro-alloys and first products thereof'},
 16: {'act': 'EU28-Aluminium and aluminium products'},
 17: {'act': 'EU28-Copper products'},
 18: {'act': 'EU28-Electricity by coal'},
 19: {'act': 'EU28-Electricity by gas'},
 20: {'act': 'EU28-Electricity by nuclear'},
 21: {'act': 'EU28-Electricity by hydro'},
 22: {'act': 'EU28-Electricity by wind'},
 23:

3. Save itemwise uncertainty values and uncertainty types (in this example, we only define 1 type in total)

In [ ]:
for i, j in zip(range(len(activities)), uncertainty_matrix):
    uncertainty_handler.metadata[i]["specific"] = j  # here "j" is a list
    uncertainty_handler.metadata[i]["type"] = uncertainty_type
uncertainty_handler.metadata[0] # now we save one uncertainty for one activity in the metadata.

{0: {'act': 'column_1',
  'specific': [1.01,
   2.02,
   3.0300000000000002,
   4.04,
   5.05,
   6.0600000000000005,
   7.07,
   8.08,
   9.09,
   10.1,
   11.11,
   12.120000000000001,
   13.13,
   14.14,
   15.15,
   16.16,
   17.17,
   18.18,
   19.19,
   20.2,
   21.21,
   22.22,
   23.23,
   24.240000000000002,
   25.25,
   26.26,
   27.27,
   28.28,
   29.29,
   30.3,
   31.31,
   32.32,
   33.33,
   34.34,
   35.35,
   36.36,
   37.37,
   38.38,
   39.39,
   40.4,
   41.410000000000004,
   42.42,
   43.43,
   44.44,
   45.45,
   46.46,
   47.47,
   48.480000000000004,
   49.49,
   50.5,
   51.51,
   52.52,
   53.53,
   54.54,
   55.55,
   56.56,
   57.57,
   58.58,
   59.59,
   60.6,
   61.61,
   62.62,
   63.63,
   64.64,
   65.65,
   66.66,
   67.67,
   68.68,
   69.69,
   70.7,
   71.71,
   72.72,
   73.73,
   74.74,
   75.75,
   76.76,
   77.77,
   78.78],
  'type': 2},
 1: {'act': 'column_2',
  'specific': [1.01,
   2.02,
   3.0300000000000002,
   4.04,
   5.05,
   6.06000

4. Get matrix data in datapackage needed format

In [41]:
(fg_bg_tech_data, fg_bg_tech_indices, fg_bg_tech_flip), (fg_bg_bio_data, fg_bg_bio_indices), (bg_cf_data, bg_cf_indices) = dp_builder.prepare_dp_matrix(fg_bg_tech_matrix, fg_bg_bio_matrix, bg_cf_matrix)

5. Form uncertainties into datapackage needed format.

Here we only add uncertainty to technosphere matrix, if you want to add also to biosphere, you can save the uncertainty data to metadata first, and use the same way to add uncertainty for biosphere.

In [42]:
fg_bg_tech_uncertainty_array = uncertainty_handler.add_ununiform_uncertainty(fg_bg_tech_data, fg_bg_tech_indices, fg_bg_tech_flip, "itemwise")

6. Create datapackage

In [ ]:
bg_dp_2 = dp_builder.prepare_datapackage([(fg_bg_tech_data, fg_bg_tech_indices, fg_bg_tech_flip), (fg_bg_bio_data, fg_bg_bio_indices), (fg_bg_cf_data, fg_bg_cf_indices)], [fg_bg_tech_uncertainty_array, None, None])
bg_dp_2.data

[array([( 0,  0), ( 1,  1), ( 2,  2), ..., (77, 75), (77, 76), (77, 77)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([1.00000000e+00, 1.00000000e+00, 7.36338343e-01, ...,
        6.24171469e-03, 9.40994608e-04, 9.95178298e-01]),
 array([(2,  0.0000000e+00, 0.00995033, nan, nan, nan, False),
        (2,  0.0000000e+00, 0.7030975 , nan, nan, nan, False),
        (2, -3.0606556e-01, 1.1085626 , nan, nan, nan, False), ...,
        (2, -5.0765004e+00, 4.366659  , nan, nan, nan, False),
        (2, -6.9685731e+00, 4.366659  , nan, nan, nan, False),
        (2, -4.8333639e-03, 4.366659  , nan, nan, nan, False)],
       dtype=[('uncertainty_type', 'u1'), ('loc', '<f4'), ('scale', '<f4'), ('shape', '<f4'), ('minimum', '<f4'), ('maximum', '<f4'), ('negative', '?')]),
 array([False, False, False, ...,  True,  True, False]),
 array([(78,  2), (78,  3), (78,  4), ..., (97, 75), (97, 76), (97, 77)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([6.92719535e+04, 2.24902521e+05, 1

5. Run simulation

In [45]:
lca = bc.LCA(
            demand={mysector : 1},
            data_objs=[bg_dp_2],
            use_distributions=True,
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, activities[mysector]}")

Brightway calculated lca score: (-73748456054.72745, 'RoW-Steam and hot water supply services')
